**Id:** 55031 **Nome:** RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI<br>
**Id:** 52662 **Nome:** CÁLCULO DIFERENCIAL E INTEGRAL I<br>
**Id:** 55022 **Nome:** FUNDAMENTOS MATEMÁTICOS DA COMPUTAÇÃO I<br>

In [33]:
import numpy as np
import pandas as pd

In [45]:
pd_disc = pd.read_csv('disciplinas.csv')
#pd_disc.iloc[0]
pd_disc

,id_disc,cod_disc,nome_disc,hora_disc
0,55031,IMD0019,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,180
1,52662,IMD0024,CÁLCULO DIFERENCIAL E INTEGRAL I,90
2,55022,IMD0028,FUNDAMENTOS MATEMÁTICOS DA COMPUTAÇÃO I,90


In [35]:
pd_turmas2017_1 = pd.read_csv('turmas-2017.1.csv', error_bad_lines = False, warn_bad_lines = False, encoding = "utf-8", sep=";", usecols=['id_turma', 'codigo_turma', 'id_docente_interno', 'id_componente_curricular', 'local', 'ano', 'periodo', 'descricao_horario', 'total_solicitacoes', 'capacidade_aluno', 'situacao_turma'])
pd_turmas2017_1.head(5)

,id_turma,codigo_turma,id_docente_interno,id_componente_curricular,local,ano,periodo,descricao_horario,total_solicitacoes,capacidade_aluno,situacao_turma
0,57593168,1,26652.0,61150,Sala 24,2017,1,2T12,NaN,50.0,CONSOLIDADA
1,57593169,2,21707.0,61150,Sala 24,2017,1,4T12,NaN,50.0,CONSOLIDADA
2,57593170,1,21707.0,61152,Sala 24,2017,1,3T56,0.0,50.0,EXCLUÍDA
3,57593172,1,5752921.0,18814,Sala 25,2017,1,2T3456,0.0,30.0,EXCLUÍDA
4,57593173,1,5752917.0,18773,LAMUCO,2017,1,3T34,24.0,20.0,CONSOLIDADA


In [39]:
pd_docentes = pd.read_csv('docentes.csv', encoding = "utf-8", sep=";", usecols=['id_servidor', 'nome', 'formacao', 'tipo_jornada_trabalho', 'vinculo', 'classe_funcional', 'lotacao'])
pd_docentes.head(5)

,id_servidor,nome,formacao,tipo_jornada_trabalho,vinculo,classe_funcional,lotacao
0,21041,GILDETE AZEVEDO,DESCONHECIDA,Dedicação exclusiva,Ativo Permanente,C ...,NÚCLEO DE EDUCAÇÃO DA INFÂNCIA
1,5753261,MARIA NUBIA PESSOA,ESPECIALIZAÇÃO,Dedicação exclusiva,Ativo Permanente,D ...,NÚCLEO DE EDUCAÇÃO DA INFÂNCIA
2,5752617,RUTILENE SANTOS DE SOUSA,MESTRADO,Dedicação exclusiva,Ativo Permanente,D ...,NÚCLEO DE EDUCAÇÃO DA INFÂNCIA
3,22092,ADAUTO CHIAMENTI,DOUTORADO,Dedicação exclusiva,Ativo Permanente,DV ...,ESCOLA AGRÍCOLA DE JUNDIAÍ
4,2263697,ADELE GUIMARAES UBARANA SANTOS,MESTRADO,Dedicação exclusiva,Ativo Permanente,DIII ...,NÚCLEO DE EDUCAÇÃO DA INFÂNCIA


In [37]:
list_turmas = pd_turmas2017_1[pd_turmas2017_1['id_componente_curricular'] == pd_disc['id_disc'][0]]
list_turmas

,id_turma,codigo_turma,id_docente_interno,id_componente_curricular,local,ano,periodo,descricao_horario,total_solicitacoes,capacidade_aluno,situacao_turma
4880,57597051,1,5757585.0,55031,B203 - CIVT,2017,1,246M3456 (13/02/2017 - 01/07/2017),22.0,83.0,CONSOLIDADA
4881,57597051,1,5756816.0,55031,B203 - CIVT,2017,1,246M3456 (13/02/2017 - 01/07/2017),22.0,83.0,CONSOLIDADA
4888,57597055,2,5756816.0,55031,B204 - CIVT,2017,1,246M3456 (13/02/2017 - 01/07/2017),16.0,81.0,CONSOLIDADA
4889,57597055,2,5759154.0,55031,B204 - CIVT,2017,1,246M3456 (13/02/2017 - 01/07/2017),16.0,81.0,CONSOLIDADA
4891,57597057,3,5756386.0,55031,B204,2017,1,246T1234 (13/02/2017 - 01/07/2017),50.0,89.0,CONSOLIDADA
4892,57597057,3,5757585.0,55031,B204,2017,1,246T1234 (13/02/2017 - 01/07/2017),50.0,89.0,CONSOLIDADA
4893,57597058,4,5758787.0,55031,B203 - CIVT,2017,1,246N1234 (13/02/2017 - 01/07/2017),25.0,87.0,CONSOLIDADA
4894,57597058,4,5757583.0,55031,B203 - CIVT,2017,1,246N1234 (13/02/2017 - 01/07/2017),25.0,87.0,CONSOLIDADA
4898,57597062,5,5757583.0,55031,B204 - CIVT,2017,1,246N1234 (13/02/2017 - 01/07/2017),30.0,77.0,CONSOLIDADA
4899,57597062,5,5757669.0,55031,B204 - CIVT,2017,1,246N1234 (13/02/2017 - 01/07/2017),30.0,77.0,CONSOLIDADA


In [47]:
newList = list_turmas.assign(nome_disc=pd_disc['nome_disc'][0])
newList


,id_turma,codigo_turma,id_docente_interno,id_componente_curricular,local,ano,periodo,descricao_horario,total_solicitacoes,capacidade_aluno,situacao_turma,nome_disc
4880,57597051,1,5757585.0,55031,B203 - CIVT,2017,1,246M3456 (13/02/2017 - 01/07/2017),22.0,83.0,CONSOLIDADA,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI
4881,57597051,1,5756816.0,55031,B203 - CIVT,2017,1,246M3456 (13/02/2017 - 01/07/2017),22.0,83.0,CONSOLIDADA,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI
4888,57597055,2,5756816.0,55031,B204 - CIVT,2017,1,246M3456 (13/02/2017 - 01/07/2017),16.0,81.0,CONSOLIDADA,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI
4889,57597055,2,5759154.0,55031,B204 - CIVT,2017,1,246M3456 (13/02/2017 - 01/07/2017),16.0,81.0,CONSOLIDADA,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI
4891,57597057,3,5756386.0,55031,B204,2017,1,246T1234 (13/02/2017 - 01/07/2017),50.0,89.0,CONSOLIDADA,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI
4892,57597057,3,5757585.0,55031,B204,2017,1,246T1234 (13/02/2017 - 01/07/2017),50.0,89.0,CONSOLIDADA,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI
4893,57597058,4,5758787.0,55031,B203 - CIVT,2017,1,246N1234 (13/02/2017 - 01/07/2017),25.0,87.0,CONSOLIDADA,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI
4894,57597058,4,5757583.0,55031,B203 - CIVT,2017,1,246N1234 (13/02/2017 - 01/07/2017),25.0,87.0,CONSOLIDADA,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI
4898,57597062,5,5757583.0,55031,B204 - CIVT,2017,1,246N1234 (13/02/2017 - 01/07/2017),30.0,77.0,CONSOLIDADA,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI
4899,57597062,5,5757669.0,55031,B204 - CIVT,2017,1,246N1234 (13/02/2017 - 01/07/2017),30.0,77.0,CONSOLIDADA,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI
